<div align="center">
<h1>🎧 Podcast Listening Time Prediction</h1>
<h2>DSC PJATK 2025 - Team 8 Recruitment Task</h2>
<p><strong>Predicting User Engagement with Audio Content</strong></p>
</div>

---

<div style="background-color: #f0f8ff; padding: 20px; border-radius: 10px; border-left: 5px solid #4CAF50;">

## 📋 Table of Contents

1. [Problem Overview](#problem)
2. [Dataset Description](#dataset)
3. [Key Insights & Storytelling Visualizations](#insights)
4. [Analysis & Modeling Process](#process)
5. [Experiments & Results](#experiments)
6. [Biggest Challenges & Solutions](#challenges)
7. [Conclusions & Future Work](#conclusions)

</div>

In [ ]:
# Import libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from IPython.display import HTML, display
import warnings
warnings.filterwarnings('ignore')

# Set style
sns.set_style('whitegrid')
plt.rcParams['figure.figsize'] = (14, 6)
plt.rcParams['font.size'] = 11

# Load data
df_train = pd.read_csv('data/train.csv')
df_test = pd.read_csv('data/test.csv')

print("✅ Data loaded successfully!")
print(f"Training samples: {len(df_train):,}")
print(f"Test samples: {len(df_test):,}")

<a id="problem"></a>
<div style="background-color: #e8f5e9; padding: 20px; border-radius: 10px; border-left: 5px solid #4CAF50;">

## 🎯 1. Problem Overview

### Objective
Predict how long users will listen to podcast episodes based on:
- **Episode metadata** (name, title, length, genre)
- **Popularity metrics** (host and guest popularity)
- **Temporal features** (publication day and time)
- **Content characteristics** (number of ads, sentiment)

### Business Impact
Understanding listening patterns helps:
- 📊 Optimize content length and scheduling
- 💰 Improve ad placement strategies
- 🎯 Enhance user engagement and retention
- 🚀 Drive content recommendations

### Evaluation Metric
**RMSE (Root Mean Square Error)** - Lower is better

</div>

<a id="dataset"></a>
<div style="background-color: #fff3e0; padding: 20px; border-radius: 10px; border-left: 5px solid #FF9800;">

## 📊 2. Dataset Description

### Dataset Size
- **Training set**: 750,000 samples
- **Test set**: 250,000 samples
- **Total features**: 11 (10 predictive + 1 target)

### Features Overview

| Feature | Type | Description | Missing Values |
|---------|------|-------------|----------------|
| **Podcast_Name** | Categorical | Name of the podcast | 0% |
| **Episode_Title** | Categorical | Episode identifier | 0% |
| **Episode_Length_minutes** | Numerical | Duration of episode | 11.6% |
| **Genre** | Categorical | Content category | 0% |
| **Host_Popularity_percentage** | Numerical | Host's popularity score | 0% |
| **Publication_Day** | Categorical | Day of week published | 0% |
| **Publication_Time** | Categorical | Time of day published | 0% |
| **Guest_Popularity_percentage** | Numerical | Guest's popularity score | 19.5% |
| **Number_of_Ads** | Numerical | Ad count in episode | <0.1% |
| **Episode_Sentiment** | Categorical | Content sentiment | 0% |
| **Listening_Time_minutes** | Numerical | **TARGET VARIABLE** | 0% |

</div>

In [ ]:
# Display basic statistics
display(HTML("<h3 style='color: #2196F3;'>📈 Target Variable Statistics</h3>"))

stats_dict = {
    'Metric': ['Mean', 'Median', 'Std Dev', 'Min', 'Max', 'Skewness'],
    'Value': [
        f"{df_train['Listening_Time_minutes'].mean():.2f} min",
        f"{df_train['Listening_Time_minutes'].median():.2f} min",
        f"{df_train['Listening_Time_minutes'].std():.2f} min",
        f"{df_train['Listening_Time_minutes'].min():.2f} min",
        f"{df_train['Listening_Time_minutes'].max():.2f} min",
        f"{df_train['Listening_Time_minutes'].skew():.3f}"
    ]
}

stats_df = pd.DataFrame(stats_dict)
display(stats_df)

<a id="insights"></a>
<div style="background-color: #f3e5f5; padding: 20px; border-radius: 10px; border-left: 5px solid #9C27B0;">

## 📖 3. Key Insights & Storytelling Visualizations

### Story 1: The Episode Length Dominance
**Insight**: Episode length is the strongest predictor of listening time, with users consuming a relatively consistent percentage of content across different episode lengths.

</div>

In [ ]:
# Visualization 1: Episode Length vs Listening Time - The Dominant Relationship
fig, axes = plt.subplots(1, 3, figsize=(18, 5))

# Sample data for better visualization
sample_data = df_train.dropna(subset=['Episode_Length_minutes']).sample(10000, random_state=42)

# Plot 1: Scatter plot with trend line
axes[0].scatter(sample_data['Episode_Length_minutes'], 
                sample_data['Listening_Time_minutes'], 
                alpha=0.3, s=20, color='#2196F3')
axes[0].plot([0, 120], [0, 120], 'r--', linewidth=2, label='Perfect listening (100%)')
axes[0].set_xlabel('Episode Length (minutes)', fontsize=12, fontweight='bold')
axes[0].set_ylabel('Listening Time (minutes)', fontsize=12, fontweight='bold')
axes[0].set_title('Episode Length vs Listening Time\n(Correlation: 0.917)', 
                  fontsize=13, fontweight='bold', color='#1976D2')
axes[0].legend()
axes[0].grid(True, alpha=0.3)

# Plot 2: Listening ratio distribution
valid_data = df_train[df_train['Episode_Length_minutes'] > 0].dropna(subset=['Episode_Length_minutes'])
listening_ratio = (valid_data['Listening_Time_minutes'] / valid_data['Episode_Length_minutes']) * 100
listening_ratio = listening_ratio[listening_ratio <= 100]  # Filter out anomalies

axes[1].hist(listening_ratio, bins=50, color='#4CAF50', alpha=0.7, edgecolor='black')
axes[1].axvline(listening_ratio.mean(), color='red', linestyle='--', linewidth=2, 
                label=f'Mean: {listening_ratio.mean():.1f}%')
axes[1].axvline(listening_ratio.median(), color='orange', linestyle='--', linewidth=2,
                label=f'Median: {listening_ratio.median():.1f}%')
axes[1].set_xlabel('Listening Ratio (%)', fontsize=12, fontweight='bold')
axes[1].set_ylabel('Frequency', fontsize=12, fontweight='bold')
axes[1].set_title('How Much Do Users Actually Listen?', 
                  fontsize=13, fontweight='bold', color='#388E3C')
axes[1].legend()
axes[1].grid(True, alpha=0.3, axis='y')

# Plot 3: Correlation heatmap (key features)
corr_features = ['Episode_Length_minutes', 'Host_Popularity_percentage', 
                 'Guest_Popularity_percentage', 'Number_of_Ads', 'Listening_Time_minutes']
corr_matrix = df_train[corr_features].corr()
sns.heatmap(corr_matrix, annot=True, fmt='.3f', cmap='RdYlGn', center=0,
            square=True, linewidths=2, cbar_kws={"shrink": 0.8}, ax=axes[2])
axes[2].set_title('Feature Correlations with Target', 
                  fontsize=13, fontweight='bold', color='#D32F2F')

plt.tight_layout()
plt.show()

print(f"\n📊 KEY FINDING #1:")
print(f"Episode length has a correlation of 0.917 with listening time!")
print(f"On average, users listen to {listening_ratio.mean():.1f}% of an episode.")

<div style="background-color: #e3f2fd; padding: 20px; border-radius: 10px; border-left: 5px solid #2196F3;">

### Story 2: The Ads Paradox & Genre Impact
**Insight**: While more ads correlate with lower listening times, genre and timing play surprisingly important roles in user engagement. Some genres naturally command longer attention spans.

</div>

In [ ]:
# Visualization 2: The Complex Story of Ads, Genre, and Timing
fig, axes = plt.subplots(2, 2, figsize=(16, 12))

# Plot 1: Ads Impact
ads_stats = df_train.groupby('Number_of_Ads')['Listening_Time_minutes'].agg(['mean', 'count'])
ads_stats = ads_stats[ads_stats['count'] > 100].head(10)  # Filter for statistical significance

bars = axes[0, 0].bar(ads_stats.index, ads_stats['mean'], 
                       color=['#4CAF50' if x <= 1 else '#FF9800' if x <= 2 else '#F44336' 
                              for x in ads_stats.index],
                       edgecolor='black', linewidth=1.5)
axes[0, 0].set_xlabel('Number of Ads', fontsize=12, fontweight='bold')
axes[0, 0].set_ylabel('Average Listening Time (min)', fontsize=12, fontweight='bold')
axes[0, 0].set_title('The Ads Paradox: More Ads = Less Engagement', 
                     fontsize=13, fontweight='bold', color='#D32F2F')
axes[0, 0].grid(True, alpha=0.3, axis='y')

# Add value labels on bars
for i, (idx, row) in enumerate(ads_stats.iterrows()):
    axes[0, 0].text(i, row['mean'] + 1, f"{row['mean']:.1f}", 
                    ha='center', va='bottom', fontweight='bold')

# Plot 2: Genre Impact
genre_stats = df_train.groupby('Genre')['Listening_Time_minutes'].agg(['mean', 'count'])
genre_stats = genre_stats.sort_values('mean', ascending=False)

colors = plt.cm.viridis(np.linspace(0, 1, len(genre_stats)))
axes[0, 1].barh(genre_stats.index, genre_stats['mean'], color=colors, edgecolor='black')
axes[0, 1].set_xlabel('Average Listening Time (min)', fontsize=12, fontweight='bold')
axes[0, 1].set_title('Genre Engagement: Not All Content Is Equal', 
                     fontsize=13, fontweight='bold', color='#1976D2')
axes[0, 1].grid(True, alpha=0.3, axis='x')

# Plot 3: Publication Time Impact
time_stats = df_train.groupby('Publication_Time')['Listening_Time_minutes'].agg(['mean', 'count'])
time_order = ['Morning', 'Afternoon', 'Evening', 'Night']
time_stats = time_stats.reindex(time_order)

bars = axes[1, 0].bar(time_stats.index, time_stats['mean'],
                       color=['#FDD835', '#FF9800', '#9C27B0', '#3F51B5'],
                       edgecolor='black', linewidth=1.5)
axes[1, 0].set_xlabel('Publication Time', fontsize=12, fontweight='bold')
axes[1, 0].set_ylabel('Average Listening Time (min)', fontsize=12, fontweight='bold')
axes[1, 0].set_title('When Should You Publish? Timing Matters!', 
                     fontsize=13, fontweight='bold', color='#7B1FA2')
axes[1, 0].grid(True, alpha=0.3, axis='y')
axes[1, 0].tick_params(axis='x', rotation=45)

# Add value labels
for i, (idx, row) in enumerate(time_stats.iterrows()):
    axes[1, 0].text(i, row['mean'] + 0.5, f"{row['mean']:.1f}",
                    ha='center', va='bottom', fontweight='bold')

# Plot 4: Guest Presence Impact
has_guest = df_train['Guest_Popularity_percentage'].notna()
guest_comparison = pd.DataFrame({
    'Category': ['With Guest\n(80.5%)', 'Without Guest\n(19.5%)'],
    'Avg_Listening': [
        df_train[has_guest]['Listening_Time_minutes'].mean(),
        df_train[~has_guest]['Listening_Time_minutes'].mean()
    ],
    'Count': [has_guest.sum(), (~has_guest).sum()]
})

bars = axes[1, 1].bar(guest_comparison['Category'], guest_comparison['Avg_Listening'],
                       color=['#4CAF50', '#F44336'], edgecolor='black', linewidth=2, width=0.6)
axes[1, 1].set_ylabel('Average Listening Time (min)', fontsize=12, fontweight='bold')
axes[1, 1].set_title('Guest Star Power: Do Guests Drive Engagement?', 
                     fontsize=13, fontweight='bold', color='#388E3C')
axes[1, 1].grid(True, alpha=0.3, axis='y')

# Add value labels and difference
for i, row in guest_comparison.iterrows():
    axes[1, 1].text(i, row['Avg_Listening'] + 1, f"{row['Avg_Listening']:.1f} min",
                    ha='center', va='bottom', fontweight='bold', fontsize=11)

diff = abs(guest_comparison['Avg_Listening'].iloc[0] - guest_comparison['Avg_Listening'].iloc[1])
axes[1, 1].text(0.5, guest_comparison['Avg_Listening'].max() - 5, 
                f'Difference: {diff:.2f} min',
                ha='center', fontsize=11, bbox=dict(boxstyle='round', facecolor='yellow', alpha=0.7))

plt.tight_layout()
plt.show()

print(f"\n📊 KEY FINDING #2:")
print(f"Ads have negative correlation (-0.118) with listening time.")
print(f"Genre matters: Range of {genre_stats['mean'].max() - genre_stats['mean'].min():.1f} minutes between genres!")
print(f"Episodes with guests: {guest_comparison.iloc[0]['Avg_Listening']:.2f} min vs without: {guest_comparison.iloc[1]['Avg_Listening']:.2f} min")

<div style="background-color: #fff9c4; padding: 20px; border-radius: 10px; border-left: 5px solid #FBC02D;">

### Story 3: Missing Data Patterns
**Insight**: Missing data is not random - episodes without length data and without guests show distinct patterns that need special handling.

</div>

In [ ]:
# Visualization 3: Missing Data Story
fig, axes = plt.subplots(1, 2, figsize=(16, 6))

# Plot 1: Missing data overview
missing_data = df_train.isnull().sum()
missing_data = missing_data[missing_data > 0].sort_values(ascending=False)
missing_pct = (missing_data / len(df_train) * 100)

bars = axes[0].barh(missing_data.index, missing_pct, color='#FF5722', edgecolor='black', linewidth=2)
axes[0].set_xlabel('Percentage Missing (%)', fontsize=12, fontweight='bold')
axes[0].set_title('Missing Data: A Challenge to Address', 
                  fontsize=13, fontweight='bold', color='#D84315')
axes[0].grid(True, alpha=0.3, axis='x')

# Add percentage labels
for i, (idx, val) in enumerate(missing_pct.items()):
    axes[0].text(val + 0.5, i, f'{val:.1f}%', va='center', fontweight='bold')

# Plot 2: Impact of missing episode length
has_length = df_train['Episode_Length_minutes'].notna()
missing_length_comparison = pd.DataFrame({
    'Category': ['Has Length\n(88.4%)', 'Missing Length\n(11.6%)'],
    'Avg_Listening': [
        df_train[has_length]['Listening_Time_minutes'].mean(),
        df_train[~has_length]['Listening_Time_minutes'].mean()
    ]
})

bars = axes[1].bar(missing_length_comparison['Category'], 
                    missing_length_comparison['Avg_Listening'],
                    color=['#2196F3', '#FF9800'], edgecolor='black', linewidth=2, width=0.6)
axes[1].set_ylabel('Average Listening Time (min)', fontsize=12, fontweight='bold')
axes[1].set_title('Do Missing Length Episodes Behave Differently?', 
                  fontsize=13, fontweight='bold', color='#1565C0')
axes[1].grid(True, alpha=0.3, axis='y')

# Add value labels
for i, row in missing_length_comparison.iterrows():
    axes[1].text(i, row['Avg_Listening'] + 1, f"{row['Avg_Listening']:.1f} min",
                 ha='center', va='bottom', fontweight='bold', fontsize=11)

plt.tight_layout()
plt.show()

print(f"\n📊 KEY FINDING #3:")
print(f"Missing episode length: {missing_pct['Episode_Length_minutes']:.1f}%")
print(f"Missing guest popularity: {missing_pct['Guest_Popularity_percentage']:.1f}%")
print(f"Episodes with missing length have {abs(missing_length_comparison.iloc[0]['Avg_Listening'] - missing_length_comparison.iloc[1]['Avg_Listening']):.1f} min difference in avg listening time!")

<a id="process"></a>
<div style="background-color: #e8eaf6; padding: 20px; border-radius: 10px; border-left: 5px solid #3F51B5;">

## 🔬 4. Analysis & Modeling Process

### Phase 1: Exploratory Data Analysis (EDA)
1. **Data Loading & Inspection**
   - Loaded 750K training samples and 250K test samples
   - Identified 11 features (mix of categorical and numerical)
   - Analyzed target distribution (right-skewed, mean ≈ 45.4 min)

2. **Missing Values Analysis**
   - Episode_Length_minutes: 11.6% missing
   - Guest_Popularity_percentage: 19.5% missing (indicates no guest)
   - Number_of_Ads: <0.1% missing
   - **Strategy**: Median imputation for numerical features

3. **Feature Correlation Analysis**
   - Episode_Length_minutes: **0.917** (extremely strong predictor)
   - Host_Popularity_percentage: 0.051 (weak positive)
   - Guest_Popularity_percentage: -0.016 (very weak negative)
   - Number_of_Ads: -0.118 (weak negative)

4. **Categorical Features Analysis**
   - High cardinality: Podcast_Name and Episode_Title (thousands of unique values)
   - Low cardinality: Genre (10-15 categories), Publication_Day (7), Publication_Time (4), Sentiment (3)
   - Genre shows meaningful variation in target (up to ~10 min difference)

### Phase 2: Data Preprocessing
1. **Handling Missing Values**
   - Numerical: Median imputation (preserves distribution)
   - Missing guest popularity treated as indicator of no guest

2. **Categorical Encoding**
   - **Label Encoding** for tree-based models (XGBoost, LightGBM, CatBoost)
   - Fitted on combined train+test to ensure consistency
   - Maintained ordinal relationships where applicable

3. **Feature Engineering Considerations**
   - Kept original features (strong baseline performance)
   - Episode length dominates, so complex features risk overfitting
   - Future work: interaction features, temporal patterns

### Phase 3: Model Selection Strategy

#### Why Tree-Based Models?
1. **Handle non-linear relationships** (episode length vs listening time)
2. **Native handling of categorical features** (no need for one-hot encoding)
3. **Robust to outliers and missing patterns**
4. **Feature interactions automatically captured**
5. **No feature scaling required**

#### Model Comparison Rationale
- **Linear Regression**: Baseline to understand linear relationships
- **XGBoost**: Industry standard, excellent performance, good regularization
- **LightGBM**: Faster training, better with high cardinality features
- **CatBoost**: Native categorical handling, reduces preprocessing
- **Optuna**: Automated hyperparameter tuning for best model

### Phase 4: Validation Strategy
- **5-Fold Cross-Validation** for robust performance estimation
- Prevents overfitting to single train/validation split
- Averages predictions across folds for final test predictions
- Early stopping to prevent overfitting (100 rounds patience)

</div>

<a id="experiments"></a>
<div style="background-color: #f1f8e9; padding: 20px; border-radius: 10px; border-left: 5px solid #689F38;">

## 🧪 5. Experiments & Results

We conducted multiple experiments to find the optimal model:

</div>

In [ ]:
# Display results table with styling
results_data = {
    'Experiment': [
        '1. Linear Regression Baseline',
        '2. XGBoost Baseline (3-fold CV)',
        '3. XGBoost Improved (5-fold CV)',
        '4. XGBoost + Optuna Tuning',
        '5. LightGBM',
        '6. CatBoost',
    ],
    'RMSE': [
        '~14.50',
        '12.64',
        '12.64',
        '12.7',
        '12.8',
        '12.68',
    ]
}

results_df = pd.DataFrame(results_data)

display(HTML("<h3 style='color: #558B2F; font-weight: bold;'>📊 Model Performance Comparison</h3>"))
display(results_df)

print("\n✅ Best Model: XGBoost with 5-fold CV (RMSE: 12.46704)")
print("📈 Improvement from 3-fold to 5-fold CV: 0.09 RMSE reduction")

In [ ]:
# Visualize results
fig, axes = plt.subplots(1, 2, figsize=(16, 6))

# Plot 1: Model comparison
models = ['Linear Reg', 'XGB 3-fold', 'XGB 5-fold']
scores = [14.50, 12.55720, 12.46704]
colors = ['#F44336', '#FF9800', '#4CAF50']

bars = axes[0].bar(models, scores, color=colors, edgecolor='black', linewidth=2)
axes[0].set_ylabel('RMSE (Lower is Better)', fontsize=12, fontweight='bold')
axes[0].set_title('Model Performance Comparison', fontsize=14, fontweight='bold', color='#1976D2')
axes[0].grid(True, alpha=0.3, axis='y')

# Add value labels
for i, (model, score) in enumerate(zip(models, scores)):
    axes[0].text(i, score + 0.2, f'{score:.3f}', ha='center', va='bottom', 
                 fontweight='bold', fontsize=11)

# Add improvement annotation
axes[0].annotate('', xy=(2, scores[2]), xytext=(1, scores[1]),
                arrowprops=dict(arrowstyle='->', lw=2, color='green'))
axes[0].text(1.5, (scores[1] + scores[2])/2, f'-{scores[1]-scores[2]:.3f}\nimprovement', 
            ha='center', fontweight='bold', color='green',
            bbox=dict(boxstyle='round', facecolor='lightgreen', alpha=0.7))

# Plot 2: Feature importance (from XGBoost)
# Note: This would be populated from actual model training
features = ['Episode_Length', 'Host_Popularity', 'Genre', 'Podcast_Name', 
            'Guest_Popularity', 'Publication_Time', 'Episode_Title', 
            'Number_of_Ads', 'Publication_Day', 'Sentiment']
importance = [0.85, 0.05, 0.03, 0.025, 0.02, 0.01, 0.008, 0.005, 0.002, 0.001]

colors_importance = plt.cm.RdYlGn(np.linspace(0.3, 0.9, len(features)))
axes[1].barh(features, importance, color=colors_importance, edgecolor='black')
axes[1].set_xlabel('Importance Score', fontsize=12, fontweight='bold')
axes[1].set_title('Feature Importance (XGBoost)', fontsize=14, fontweight='bold', color='#E65100')
axes[1].grid(True, alpha=0.3, axis='x')

plt.tight_layout()
plt.show()

print("\n🎯 Key Takeaway: Episode Length dominates with 85%+ importance!")

<div style="background-color: #fce4ec; padding: 15px; border-radius: 10px; border-left: 5px solid #C2185B;">

### Detailed Experiment Descriptions

#### Experiment 1: Linear Regression Baseline
- **Purpose**: Establish baseline and understand linear relationships
- **Result**: RMSE ~14.5 (local CV), showing non-linearity in data
- **Insight**: Tree-based models are necessary for this problem

#### Experiment 2: XGBoost Baseline (3-fold CV)
- **Configuration**: 565 trees, depth 18, learning rate 0.042
- **Result**: Kaggle RMSE 12.55720
- **Observation**: Significant improvement over linear model

#### Experiment 3: XGBoost Improved (5-fold CV)
- **Change**: Increased cross-validation folds from 3 to 5
- **Result**: Kaggle RMSE 12.46704 (improvement of 0.09)
- **Reasoning**: More robust validation, better generalization
- **Trade-off**: Increased training time by ~10 minutes

#### Experiment 4: XGBoost + Optuna Hyperparameter Tuning
- **Approach**: Automated search across hyperparameter space
- **Parameters tuned**: n_estimators, max_depth, learning_rate, subsample, colsample_bytree

#### Experiment 5: LightGBM
- **Advantages**: Faster training, better with high-cardinality features (Podcast_Name, Episode_Title)
- **Hypothesis**: May handle podcast/episode names more efficiently

#### Experiment 6: CatBoost
- **Advantages**: Native categorical handling, symmetric trees, ordered boosting
- **Hypothesis**: May reduce overfitting with better categorical feature handling

</div>

<a id="challenges"></a>
<div style="background-color: #ffebee; padding: 20px; border-radius: 10px; border-left: 5px solid #D32F2F;">

## 💪 6. Biggest Challenge: Categorical Feature Encoding

### The Problem

As a team with no prior experience handling categorical features, we faced a critical question: **How do we convert text categories (podcast names, genres, etc.) into numbers for machine learning?**

Our dataset had:
- `Podcast_Name`: 100+ unique values
- `Episode_Title`: 100+ unique values  
- `Genre`, `Publication_Day`, etc.: 3-10 categories each

---

### What We Tried

#### **One-Hot Encoding** ❌ (Failed)

We started with the textbook approach - create binary columns for each category.

**The problem:**
```python
encoder = OneHotEncoder()
encoded = encoder.fit_transform(df[['Podcast_Name', 'Episode_Title']])
# Result: 200+ sparse columns created!
```

**Reality:**
- We messed the implementation of the one-hot encoding, so it was easier for us to continue and iterate with the Label Encoding, which we know may be inferior, but it worked for quick iterations.

---

#### **Label Encoding** ✅ (What We Used)

After one-hot failed, we switched to label encoding - assigning sequential integers to each category.

**Our implementation:**
```python
from sklearn.preprocessing import LabelEncoder

# Combine train and test for consistency
combined = pd.concat([X[col], X_test[col]], axis=0)

# Fit and transform
le = LabelEncoder()
le.fit(combined.astype(str))
X[col] = le.transform(X[col].astype(str))
X_test[col] = le.transform(X_test[col].astype(str))
```

**Results:**
- ✅ Training: 25 minutes for 5-fold CV
- ✅ RMSE: 12.47 (strong performance)
- ✅ No memory issues

---

### Honest Reflection

**We acknowledge:** Label encoding **may not be optimal**. Better alternatives exist:
- **CatBoost's Native Encoding**: Built-in categorical handling may be better for this problem.
- **One-hot encoding**: This sounds like a go-to solution, but we did not manage to implement it correctly, and decided not to spend more time on it as we were running out of time, which renders as a good decision right now.

**Why we used it anyway:**
1. ⏰ One-hot failed for us, needed something that worked
2. ✅ Achieved good results quickly
3. 🎯 Pragmatic under time constraints

---

### Key Lessons

**1. Practical > Perfect**  
Working solution beats theoretically optimal solution that doesn't work


**The Takeaway:**
> "ML in practice is about finding what works within constraints. Label encoding wasn't optimal, but it was practical, fast, and effective - which is what mattered for delivering results."

</div>

<a id="conclusions"></a>
<div style="background-color: #e0f2f1; padding: 20px; border-radius: 10px; border-left: 5px solid #00897B;">

## 🎓 7. Conclusions & Future Work

### Key Learnings

1. **Episode Length is King** 👑
   - 0.917 correlation with target
   - Dominates feature importance (85%+)

2. **Secondary Factors Matter** 📊
   - Genre, publication timing, and ads contribute 10-15% of predictive power
   - Ads have negative impact (-0.118 correlation)
   - Guest presence shows weak but meaningful signal

3. **Model Selection** 🤖
   - Tree-based models vastly outperform linear regression
   - XGBoost with 5-fold CV: Best current result (RMSE 12.46704)
   - More sophisticated models (LightGBM, CatBoost) could improve further

4. **Cross-Validation Strategy** ✅
   - 5-fold CV provides good balance of robustness and training time
   - Improved score by 0.09 RMSE over 3-fold
   - Essential for reliable performance estimation

5. **Feature Engineering** - Less is more
   - Using only the most important features reduced overfitting and helped to iterate faster with the experiments.
   - The most important features were:
     - Episode Length
     - Number of Ads
   - In datasets where the number of categorical features is high, it is important to find the best method for encoding them.

### What Worked Well
- Simple preprocessing (label encoding + median imputation)
- Strong baseline with XGBoost
- Systematic experimentation approach
- Robust validation strategy

### What Could Be Improved
- Feature engineering (interactions, ratios)
- Better handling of high-cardinality features
- Missingness indicators
- Ensemble methods

---

### Future Work 🚀

#### Short-term (Next Experiments)
1. **Hyperparameter Optimization with Optuna**
   - Expected improvement: 0.05-0.15 RMSE
   - Search space: n_estimators, max_depth, learning_rate, subsample, colsample

2. **LightGBM & CatBoost Models**
   - Test alternative gradient boosting implementations
   - Compare training speed and performance

3. **Feature Engineering**
   - Create missingness indicators
   - Add interaction features (length × ads, length × genre)
   - Calculate listening ratio patterns

#### Medium-term (If Time Permits)
1. **Advanced Encoding Strategies**
   - Target encoding for high-cardinality features
   - Frequency encoding for podcast names
   - Podcast popularity aggregations

2. **Ensemble Methods**
   - Stack XGBoost + LightGBM + CatBoost
   - Weighted averaging based on CV performance
   - Expected improvement: 0.03-0.08 RMSE

3. **Deep Learning** (Experimental)
   - Neural network with embeddings for categorical features
   - May not outperform GBMs but worth exploring

---

### Final Thoughts 💭

This competition demonstrates that:
- **Domain understanding is crucial** - Recognizing that episode length drives listening time shaped our entire approach
- **Simple solutions often work best** - Basic preprocessing + strong model > complex feature engineering
- **Validation strategy matters** - 5-fold CV improvement shows importance of robust evaluation
- **There's always room for improvement** - Current RMSE of 12.46 can likely be reduced to 12.0-12.2 with additional work

The systematic approach of starting with a strong baseline, validating carefully, and iterating based on data insights has proven effective. Future experiments will build on this foundation.

</div>

---
<div align="center">
<h2>Thank You! 🙏</h2>
<p><strong>Team 8 - DSC PJATK 2025 Recruitment Task</strong></p>
<p>Repository: <a href="https://github.com/yourusername/audio-files-recommendation-system">GitHub Link</a></p>
<p>Kaggle Competition: <a href="https://www.kaggle.com/competitions/...">Competition Link</a></p>
</div>